In [1]:
import tensorflow  as tf

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
v = tf.Variable(0.0)

In [4]:
(v + 1).numpy()

1.0

In [5]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [6]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [7]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [9]:
v.read_value()

<tf.Tensor: id=22, shape=(), dtype=float32, numpy=6.0>

In [10]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [11]:
grad = t.gradient(loss, w)

In [12]:
grad

<tf.Tensor: id=39, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [13]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [14]:
dloss_dw = t.gradient(loss, w)

In [15]:
dloss_dw

<tf.Tensor: id=46, shape=(), dtype=float32, numpy=6.0>

In [16]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [17]:
dy_dw = t.gradient(y, w)

In [18]:
dy_dw

<tf.Tensor: id=54, shape=(), dtype=float32, numpy=6.0>

In [19]:
dz_dw = t.gradient(z, w)

In [20]:
dz_dw

<tf.Tensor: id=62, shape=(), dtype=float32, numpy=108.0>

In [64]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

In [22]:
train_image.shape

(60000, 28, 28)

In [23]:
train_image = tf.expand_dims(train_image, -1)

In [65]:
test_image = tf.expand_dims(test_image, -1)

In [24]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [25]:
train_image = tf.cast(train_image/255, tf.float32)

In [66]:
test_image = tf.cast(test_image/255, tf.float32)

In [26]:
train_labels = tf.cast(train_labels, tf.int64)

In [67]:
test_labels = tf.cast(test_labels, tf.int64)

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [68]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [28]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [29]:
dataset = dataset.shuffle(10000).batch(32)

In [69]:
test_dataset = test_dataset.batch(32)

In [30]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [32]:
optimizer = tf.keras.optimizers.Adam()

In [33]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
features, labels = next(iter(dataset))

In [35]:
features.shape

TensorShape([32, 28, 28, 1])

In [36]:
labels.shape

TensorShape([32])

In [37]:
predictions = model(features)

In [38]:
predictions.shape

TensorShape([32, 10])

In [39]:
tf.argmax(predictions, axis=1)

<tf.Tensor: id=197, shape=(32,), dtype=int64, numpy=
array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 6, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=int64)>

In [40]:
labels

<tf.Tensor: id=179, shape=(32,), dtype=int64, numpy=
array([9, 5, 0, 7, 6, 6, 0, 6, 2, 3, 0, 2, 8, 1, 4, 8, 2, 7, 1, 0, 6, 1,
       5, 8, 0, 8, 3, 5, 6, 1, 2, 2], dtype=int64)>

In [41]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [70]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [61]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [71]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [72]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                              test_loss.result(),
                                                              test_accuracy.result()))
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [73]:
train()

Epoch0 loss is 0.2826021611690521, accuracy is 0.9087166786193848
Epoch 0 test_loss is 0.2495417296886444, test_accuracy is 0.91839998960495
Epoch1 loss is 0.2627265155315399, accuracy is 0.9150833487510681
Epoch 1 test_loss is 0.2302085906267166, test_accuracy is 0.9244999885559082
Epoch2 loss is 0.2477354109287262, accuracy is 0.919950008392334
Epoch 2 test_loss is 0.21737165749073029, test_accuracy is 0.9296000003814697
Epoch3 loss is 0.2352440059185028, accuracy is 0.9248166680335999
Epoch 3 test_loss is 0.20951487123966217, test_accuracy is 0.933899998664856
Epoch4 loss is 0.22479121387004852, accuracy is 0.9279166460037231
Epoch 4 test_loss is 0.200822651386261, test_accuracy is 0.9359999895095825
Epoch5 loss is 0.21607279777526855, accuracy is 0.9308666586875916
Epoch 5 test_loss is 0.19341905415058136, test_accuracy is 0.9376999735832214
Epoch6 loss is 0.2085001915693283, accuracy is 0.9337999820709229
Epoch 6 test_loss is 0.18873530626296997, test_accuracy is 0.940199971199035

KeyboardInterrupt: 

### tf.keras.metrics 汇总计算模块

In [45]:
m = tf.keras.metrics.Mean('acc')

In [46]:
m(10)

<tf.Tensor: id=222, shape=(), dtype=float32, numpy=10.0>

In [47]:
m(20)

<tf.Tensor: id=232, shape=(), dtype=float32, numpy=15.0>

In [49]:
m.result().numpy()

15.0

In [50]:
m([30, 40])

<tf.Tensor: id=250, shape=(), dtype=float32, numpy=25.0>

In [51]:
m.result()

<tf.Tensor: id=254, shape=(), dtype=float32, numpy=25.0>

In [52]:
m.reset_states()

In [54]:
m(1)

<tf.Tensor: id=266, shape=(), dtype=float32, numpy=1.0>

In [55]:
m(2)

<tf.Tensor: id=276, shape=(), dtype=float32, numpy=1.5>

In [56]:
m.result().numpy()

1.5

In [57]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [58]:
a(labels, model(features))

<tf.Tensor: id=324, shape=(), dtype=float32, numpy=0.03125>

In [59]:
1/32

0.03125

In [ ]:
#train()